In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

### Data оруулах

In [3]:
df = pd.read_csv('input/ads_data_2025-06-19_allsheet.csv')


### Оффисын нэрээр дундаж үнэ болон зарын тоог харуулах

In [4]:
# mytitle дээр бүлэглээд дундаж үнэ ба зарын тоог хамт гаргах
summary = df.groupby('mytitle').agg(
    avg_price_m2=('price_m2', 'mean'),
    ad_count=('price_m2', 'count')
).sort_values('avg_price_m2', ascending=False)

# Хамгийн өндөр үнэтэй 10 оффисыг харах
summary.tail(10)

,avg_price_m2,ad_count
mytitle,,
Titan Plus,4.551429,7
Sydney tower,4.500000,2
Хос даль,4.500000,2
Метро бизнес центр,4.470000,1
Москва төв,4.263333,3
Titan Center,4.240000,2
Хоймор оффис,3.985000,4
Dragon center,3.980000,1
Union Building,3.945714,14


In [7]:
df1 = pd.read_excel('D:/Projects/Office/Office аргачлал.xlsx', sheet_name=4)

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   №                190 non-null    float64
 1   district         193 non-null    object 
 2   khoroo           193 non-null    int64  
 3   number           187 non-null    object 
 4   zip code         165 non-null    float64
 5   Хаяг нэгтгэсэн   0 non-null      float64
 6   office_name      184 non-null    object 
 7   Бүтээц           193 non-null    object 
 8   floor            193 non-null    int64  
 9   op_date          183 non-null    object 
 10  Гудамжны нэр     187 non-null    object 
 11  Координат        182 non-null    object 
 12  Үнэлгээ          2 non-null      float64
 13  Үнэлгээ
2024.12  2 non-null      float64
 14  Large            2 non-null      float64
 15  Байрны дугаар    0 non-null      float64
dtypes: float64(7), int64(2), object(7)
memory usage: 24.3+ KB


In [ ]:
# 1. df-ээс mytitle бүрийн дундаж үнэ гаргаж dict болгон хувиргана
avg_price_dict = df.groupby('mytitle')['price_m2'].mean().to_dict()

# 2. df1['office_name']-ийг ашиглан дундаж үнийг 'Үнэлгээ' баганад оруулах
df1['Үнэлгээ'] = df1['office_name'].map(avg_price_dict)
df1.to_excel('average_price.xlsx', index=False)

In [25]:
# mytitle дээр бүлэглээд дундаж үнэ ба зарын тоог хамт гаргах
summary = df.groupby('mytitle').agg(
    avg_price_m2=('price_m2', 'min'),
    ad_count=('price_m2', 'count')
).sort_values('avg_price_m2', ascending=False)

# Хамгийн өндөр үнэтэй 10 оффисыг харах
summary.head(10)

,avg_price_m2,ad_count
mytitle,,
Minister Tower,13.47,4
Park place,13.00,1
Ayud tower,12.00,1
MERU tower,10.65,1
Paragon,10.00,1
Galaxy tower,10.00,2
Gem castle,9.80,9
Soyombo tower,9.60,17
Sumo tower,9.50,2
